In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [2]:
city = 'chicago'

In [3]:
#read in labels and ground truth data 
gt = pd.read_csv('../../../data/' + city + '/processed-labels/test_set_' + city + '.csv')

In [4]:
#split correct and incorrect labels
correct = gt[gt['verified'] == 1]
incorrect = gt[gt['verified'] == 0]

In [7]:
way_type_list = gt['way_type'].unique().tolist()

## Distance to intersections

In [5]:
#select label_type = CurbRamp and label_type = NoCurbRamp
all_curbs_correct = correct[correct['label_type'].isin(['CurbRamp', 'NoCurbRamp'])]
all_curbs_incorrect = incorrect[incorrect['label_type'].isin(['CurbRamp', 'NoCurbRamp'])]

In [8]:
for i in way_type_list:
    sns.histplot(data=all_curbs_correct.loc[(all_curbs_correct['way_type']== i)], x="distance_to_intersection", bins=100, kde=True, color="green")
    sns.histplot(data=all_curbs_incorrect.loc[(all_curbs_incorrect['way_type']== i)], x="distance_to_intersection", bins=100, kde=True, color="red")
    plt.title('Curb and Missing Curb labels distance to ' + i + ' intersection')
    plt.savefig('../../../figures/' + city + '/all_curbs_distance_to_' + i + '_intersection.png')
    plt.close() 

In [6]:
no_curb_correct = correct[correct['label_type'] == 'NoCurbRamp']
no_curb_incorrect = incorrect[incorrect['label_type'] == 'NoCurbRamp']

In [7]:
no_curb_incorrect

,label_id,gsv_panorama_id,label_type,severity,geometry,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag_list,description,tag_count,verified
83,201,DGa0dRGy0ruPxFv3516PqA,NoCurbRamp,4.0,POINT (-87.66569519042969 41.58452224731445),3,297.763397,-13.037947,359.465729,0.536690,3ca93bc2-09a8-41ac-96c1-4ada72269b1f,41.584522,-87.665695,9204,0,1,residential,14.357397,42.710998,1.0,0.0,1.0,0
533,916,WBdIPT9zZldcuXC0JhLMGA,NoCurbRamp,5.0,POINT (-87.98518371582031 42.084686279296875),1,108.062500,-11.750000,107.613297,0.771973,814f4169-98a1-4afa-80da-3b46be1da405,42.084686,-87.985184,10014,0,1,residential,34.340023,157.777684,1.0,0.0,1.0,0
534,920,dYHZD3Tb88u07W4n1kYQeg,NoCurbRamp,3.0,POINT (-87.97821044921875 42.08094024658203),1,231.062500,-11.937500,90.410858,0.929543,814f4169-98a1-4afa-80da-3b46be1da405,42.080940,-87.978210,10103,0,1,tertiary,22.429504,47.618468,1.0,0.0,1.0,0
556,1066,YmQBlI70ChnP1c1segBeVg,NoCurbRamp,4.0,POINT (-87.9853744506836 42.08518981933594),1,141.000000,-6.500000,65.854912,0.429543,814f4169-98a1-4afa-80da-3b46be1da405,42.085190,-87.985374,9960,0,1,primary,22.765617,29.138498,1.0,0.0,1.0,0
558,1015,fP9aZeEYhR67GWbh_wrwIg,NoCurbRamp,3.0,POINT (-87.97943878173828 42.07964324951172),1,324.125000,-17.375000,179.649628,0.834366,814f4169-98a1-4afa-80da-3b46be1da405,42.079643,-87.979439,10048,1,2,residential,21.109306,295.173933,1.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,20313,1ni-bVLHeAGE8Bl_3F2yMA,NoCurbRamp,5.0,POINT (-87.64288330078125 41.61532211303711),2,52.625000,-9.062500,179.045761,0.284060,44116ebd-4409-4ce5-84fa-3fed61f84d13,41.615322,-87.642883,9039,0,1,residential,20.211400,28.664997,0.0,0.0,0.0,0
5534,20621,5-VDkehRBazLcBNw501giA,NoCurbRamp,5.0,POINT (-87.64115142822266 41.58823013305664),1,136.000000,-10.000000,354.960999,0.515114,44116ebd-4409-4ce5-84fa-3fed61f84d13,41.588230,-87.641151,8965,0,1,residential,11.813250,31.034040,1.0,0.0,1.0,0
5582,20857,OAXKLZnbGSrnWz4Z2vlx_Q,NoCurbRamp,5.0,POINT (-87.64935302734375 41.61540603637695),2,293.546875,-14.640625,271.772705,0.370520,44116ebd-4409-4ce5-84fa-3fed61f84d13,41.615406,-87.649353,8848,0,1,residential,17.861818,35.135480,1.0,0.0,1.0,0
5583,20858,OAXKLZnbGSrnWz4Z2vlx_Q,NoCurbRamp,5.0,POINT (-87.64942169189453 41.615440368652344),2,293.546875,-14.640625,271.772705,0.370520,44116ebd-4409-4ce5-84fa-3fed61f84d13,41.615440,-87.649422,8830,0,1,residential,11.582122,32.527325,1.0,0.0,1.0,0


In [ ]:
#label_id to list


## Distance to roads

In [9]:
obstacles_correct = correct[correct['label_type'] == 'Obstacle']
obstacles_incorrect = incorrect[incorrect['label_type'] == 'Obstacle']
for i in way_type_list:
    sns.histplot(data=obstacles_correct.loc[(obstacles_correct['way_type']== i)], x="distance_to_road", bins=100, kde=True, color="green")
    sns.histplot(data=obstacles_incorrect.loc[(obstacles_incorrect['way_type']== i)], x="distance_to_road", bins=100, kde=True, color="red")
    plt.title('Obstacle labels distance to ' + i )
    plt.savefig('../../../figures/' + city + '/obstacle_distance_to_' + i + '.png')
    plt.close() 

In [10]:
surfaceproblems_correct = correct[correct['label_type'] == 'SurfaceProblem']
surfaceproblems_incorrect = incorrect[incorrect['label_type'] == 'SurfaceProblem']
for i in way_type_list:
    sns.histplot(data=surfaceproblems_correct.loc[(surfaceproblems_correct['way_type']== i)], x="distance_to_road", bins=100, kde=True, color="green")
    sns.histplot(data=surfaceproblems_incorrect.loc[(surfaceproblems_incorrect['way_type']== i)], x="distance_to_road", bins=100, kde=True, color="red")
    plt.title('Surface Problem labels distance to ' + i )
    plt.savefig('../../../figures/' + city + '/surface_distance_to_' + i + '.png')
    plt.close() 

## Clusters

In [12]:
#put gt unique label_type values to a list
label_types = gt['label_type'].unique().tolist()

In [13]:
for i in label_types:
    sns.countplot(data=gt.loc[(gt['label_type']== i)], x="cluster_label_count", hue='verified')
    plt.title('Number of labels within a cluster for ' + i )
    plt.savefig('../../../figures/' + city + '/cluster_label_count_' + i + '.png')
    plt.close() 